# Keyword Extraction

- Keyword Extraction ist die Extraktion von relevanten Schlüsselwörtern aus Dokumenten in Textform.
- Eine Sammlung aus Dokumenten wird dabei als Korpus bezeichnet.  

In [33]:
# Importiere benötigte Bibliotheken
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import operator
import pandas as pd
from nltk.corpus import stopwords
from nltk import word_tokenize
import spacy

In [34]:
# Definiere einen Textkorpus
corpus = [
    'Das ist das erste Dokument. Das Thema heute ist die Vorverarbeitung von Texten. Die Vorverarbeitung ist ein wichtiges Konzept der Sprachverarbeitung.',
    'Dieses Dokument ist das zweite Dokument. Mit diesem Dokument soll ebenfalls gearbeitet werden.',
    'Dies ist das dritte Dokument. Somit sind mindestens drei Dokumente vorhanden.',
    'Ist dieses Dokument das Erste?'
]

## Term Frequency (Vorkommenhäufigkeit)
- Gibt an, wie häufig ein Ausdruck in einem Dokument vorkommt

In [35]:
# Erstelle einen Vektor für die Vorkommenshäufigkeiten
vectorizer = CountVectorizer(lowercase=False)
X = vectorizer.fit_transform(corpus)
feature_names = vectorizer.get_feature_names_out()
term_frequencies = X.toarray()
# Übertrage Ergebnisse in ein Dictionary und dann DataFrame
data = {'keyword' : feature_names, 'frequency': term_frequencies[0]}
df = pd.DataFrame.from_dict(data)
df = df.sort_values(by=['frequency'], ascending=False)
df.reset_index(drop=True)

,keyword,frequency
0,ist,3
1,Das,2
2,Vorverarbeitung,2
3,ein,1
4,heute,1
5,Die,1
6,die,1
7,der,1
8,das,1
9,Thema,1


## Frage: Welche Probleme sind bei dieser Auswertung erkennbar? 
(Lösung: Folie 18)


## Das Tf-idf-Maß
- Geeigneteres Bewertungsschema als die einfache Vorkommenshäufigkeit
- Statistisches Maß zur Beurteilung der Relevanz von Termen in Dokumenten eines Korpus
- Setzt sich aus term frequency (Vorkommenshäufigkeit) und inverse document frequency (Inverse Dokumentenhäufigkeit) zusammen

In [36]:
def get_tfidf_rankings(corpus):
    # Bilde einen Tfidf Vektor
    vectorizer = TfidfVectorizer(lowercase=False)
    X = vectorizer.fit_transform(corpus)
    feature_names = vectorizer.get_feature_names_out()
    term_frequencies = X.toarray()
    # Übertrage Ergebnisse in ein Dictionary und dann Dataframe
    data = {'keyword' : feature_names, 'tfidf_score': term_frequencies[0]}
    df = pd.DataFrame.from_dict(data)
    df = df.sort_values(by=['tfidf_score'], ascending=False)
    df = df.reset_index(drop=True).round(2)
    return df

get_tfidf_rankings(corpus)

,keyword,tfidf_score
0,Das,0.41
1,Vorverarbeitung,0.41
2,ist,0.39
3,ein,0.20
4,heute,0.20
5,Die,0.20
6,die,0.20
7,der,0.20
8,Thema,0.20
9,Texten,0.20


## Vorverarbeitung (Preprocessing):
- Ziel: Reduktion der Dimensionen und entfernen von fehlerhaften Zeichen etc.
- Groß-/Kleinschreibung sollte unerheblich sein, da sie keinen Einfluss auf die Semantik hat ("Das"/"das" oder "Erste"/"erste") 
- Unwichtige Wörter wie z.B. "Der" oder "Die" könnten rausgefilter werden


### Lemmatisierung (Lemmatization)
- Grundform eines Wortes, die im Lexikon abgebildet ist
- Hilfreich für Dimensionsreduktion

In [37]:
def lemmatize(document):
    nlp = spacy.load('de_core_news_sm')
    lemmatized = ""
    document = nlp(document)
    for word in document:
        lemmatized += word.lemma_ + " "
    return lemmatized

corpus_lemmatized = []
for document in corpus:
    lemmatized = lemmatize(document)
    print(lemmatized)
    corpus_lemmatized.append(lemmatized)

der sein der erster Dokument -- der Thema heute sein der Vorverarbeitung von Text -- der Vorverarbeitung sein ein wichtig Konzept der Sprachverarbeitung -- 
dieser Dokument sein der zweiter Dokument -- mit dieser Dokument sollen ebenfalls arbeiten werden -- 
dieser sein der dritter Dokument -- somit sein mindestens drei Dokument vorhanden -- 
sein dieser Dokument der erster -- 


In [38]:
corpus = corpus_lemmatized
get_tfidf_rankings(corpus)

,keyword,tfidf_score
0,der,0.62
1,Vorverarbeitung,0.40
2,sein,0.31
3,ein,0.20
4,Konzept,0.20
5,Sprachverarbeitung,0.20
6,Text,0.20
7,Thema,0.20
8,wichtig,0.20
9,von,0.20


### Entfernen von Stoppwörtern (Stopwords)
- Stoppwörter tragen nicht zur Bedeutung / Semantik der Sätze bei
- Stoppwörter sind Füllwörter wie Artikel "der, die, das" oder "und, ist" etc.
- Da sie keine Relevanz in Bezug auf die Extraktion von Schlüsselwörtern haben, können sie herausgefiltert werden 

In [39]:
def remove_stop_words(document):
    stop_words = stopwords.words('german') # lädt Liste mit deutschen Stoppwörtern
    tokenized = word_tokenize(document)
    document_filtered = ""
    for word in tokenized:
        if word not in stop_words and word.isalpha(): # filtert Stoppwörter und Zeichen, die nicht im Alphabet sind raus
            document_filtered += word + " "
    return document_filtered

corpus_filtered = []
for document in corpus:
    print(document)
    corpus_filtered.append(remove_stop_words(document))

der sein der erster Dokument -- der Thema heute sein der Vorverarbeitung von Text -- der Vorverarbeitung sein ein wichtig Konzept der Sprachverarbeitung -- 
dieser Dokument sein der zweiter Dokument -- mit dieser Dokument sollen ebenfalls arbeiten werden -- 
dieser sein der dritter Dokument -- somit sein mindestens drei Dokument vorhanden -- 
sein dieser Dokument der erster -- 


In [40]:
corpus = corpus_filtered
get_tfidf_rankings(corpus)
print(corpus)

['erster Dokument Thema heute Vorverarbeitung Text Vorverarbeitung wichtig Konzept Sprachverarbeitung ', 'Dokument zweiter Dokument Dokument sollen ebenfalls arbeiten ', 'dritter Dokument somit mindestens drei Dokument vorhanden ', 'Dokument erster ']


### Lowercasing (Kleinschreibung)
- Vereinheitlichung der Begriffe durch Kleinschreibung

In [41]:
corpus_lowercase = []
corpus_lowercase = list(map(lambda x: x.lower(), corpus))
for document in corpus_lowercase:
    print(document)

erster dokument thema heute vorverarbeitung text vorverarbeitung wichtig konzept sprachverarbeitung 
dokument zweiter dokument dokument sollen ebenfalls arbeiten 
dritter dokument somit mindestens drei dokument vorhanden 
dokument erster 


In [42]:
corpus = corpus_lowercase
get_tfidf_rankings(corpus)

,keyword,tfidf_score
0,vorverarbeitung,0.61
1,sprachverarbeitung,0.30
2,wichtig,0.30
3,heute,0.30
4,konzept,0.30
5,thema,0.30
6,text,0.30
7,erster,0.24
8,dokument,0.16
9,arbeiten,0.00
